# Imports

In [1]:
import numpy as np
import sys
import os
#%load_ext autoreload

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#from src.utils.audio_to_spectrograms import *
from src.models.yamnet_train import *

if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    path_prefix = ''
else:
    path_prefix = '..'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

2025-12-04 01:08:04.468080: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 01:08:04.486458: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 01:08:05.167535: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 01:08:20.325328: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

Using: cpu


In [2]:
# manual train test split (stratified)
np.random.seed(0)
data = pickle.load(open('data/processed/yamnet/spectrograms_train.pkl', 'rb'))
train_size = 0.8
train_idx = []
for label in np.unique(data['event_label']):
    choices = np.where(data['event_label'] == label)[0]
    train_idx.append(np.sort(np.random.choice(choices, size = int(np.round(len(choices)*train_size)), replace = False)))
train_idx = np.sort(np.concatenate(train_idx))
len(train_idx)

2001

In [ ]:
# uncomment to retrain
"""solver = Solver(epochs = 5, train_idx=train_idx, path_prefix = path_prefix, device = device)
solver.train()"""

In [ ]:
test_path = Path("data") / "processed" / "yamnet" / "spectrograms_test.pkl"
test_data = pickle.load(open(test_path, "rb"))
filepaths = [os.path.join(DETECTION_TEST_PATH, file) for file in test_data['files']]
events = run_yamnet(filepaths)